# Análise resultado BPZ para amostra VALID DES - SV calculado com a  MAGDETMODEL

A ideia principal será analisar os primeiros resultados rodando o algoritmo BPZ - Bayesian Photometric Redshifts após o cálculo fotométrico do redshift utilizando a amostra valid. A amostra consta com dados de observações de aproximadamente 5800 galáxias.

obs: importante ressaltar que a MAGDETMODEL é calculada a partir do formato da galáxia que melhor se encaixa em um filtro que se chama SEXtractor para depois ser medido o fluxo para cada banda de acordo com o formato que se encaixa melhor.

## 0 - Setup 
Importando as bibliotecas que serão utilizadas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1 - Leitura de arquivos

Aqui é feito a leitura dos arquivos que vamos utilizar. Tanto da amostra original, do nosso resultado de output pelo algoritmo quanto auxiliares

In [ ]:
headers=[
    "u_auto", "u_auto_err", "u_det", "u_det_err",
    "g_auto", "g_auto_err", "g_det", "g_det_err", 
    "r_auto", "r_auto_err", "r_det", "r_det_err", 
    "i_auto", "i_auto_err", "i_det", "i_det_err", 
    "z_auto", "z_auto_err", "z_det", "z_det_err", 
    "Y_auto", "Y_auto_err", "Y_det", "Y_det_err", 
    "z", "ra", "dec", "z_flags", "field_id", "number"
]

original_df = pd.read_csv("../../Amostras/des_sv_main_standard.valid", delimiter=" ", names=headers)
weight_df = np.loadtxt("../../Amostras/des_sv_main_standard_weights", delimiter=" ", usecols=[2])

In [ ]:
headers_bpz = ["id", "zb", "zbmin", "zbmax", "tb", "odds", "zml", "tml", "chisq", "zspec", "M0", "other", "chisq2", "nfdet", "nfobs"]

bpz_df = pd.read_csv("main_valid_magdet_bpz.cat", delim_whitespace=True, names=headers_bpz, skiprows=17)
bpz_df.head()

## 2 - Análises 

### 2.1 - Bias e Scatter

aqui declaramos uma função para auxiliar para criar um plot que do resultado de redshift da spectroscopia vs o resultado obtido para fotometria pelo algoritmo.

In [ ]:
plt.style.use('seaborn-dark-palette')

def pz_zspec(df):
    plt.plot(df["zspec"], df["zb"], f".", color="midnightblue", alpha=0.5)
    plt.plot([0, 2], [0, 2], 'r-')

    plt.ylabel("photo-z", fontsize=18)
    plt.xlabel("zspec", fontsize=18)

    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.xlim(0, 1.5)
    plt.ylim(0, 1.5)

A plotagem do gráfico de zspec e zb é importante, pois consegue-se observar, pela distribuição dos pontos, o bias e scatter do algoritmo com a amostra.

Sendo o _bias_ um viés, que indica uma tendencia de desvio da linha vermelha, que no caso indica um cenário perfeito, onde o resultado do algoritmo para photoz é igual ao specz. E o _scatter_ sendo a dispersão da nuvem de pontos.

In [ ]:
bpz_sem_u_band_df = pd.read_csv("main_valid_magdet_bpz_test1.cat", delim_whitespace=True, names=headers_bpz, skiprows=17)

In [ ]:
plt.figure(figsize=[12,6])
plt.suptitle("photo-z vs zspec (magdet)", fontsize=16)

plt.subplot(1,2,1)
pz_zspec(bpz_sem_u_band_df)

plt.subplot(1,2,2)
pz_zspec(bpz_df)

plt.tight_layout()

discussão:

Como podemos observar, houve a reprodução muito parecida com a do artigo, podendo notar como a primeira figura se assemelha ao formato de uma espada, sendo por conta do erro que não processar o algoritmo com a banda u pode causar, já o segundo vemos que uma bainha desaparece, justamente por conter a banda u para desempatar possiveis galáxias encaixadas no algoritmo.

### 2.2 - Comparação com a disperção do zspec

Weinght - a pesagem é uma técnica que associa um peso para cada galáxia. Esse peso serve pra que quando se observar a distribuição da fotometria se chegue mais perto do exemplo real.

A ideia é que nesses dois gráficos a seguir a gente possa comparar como o a técnica de pesagem impacta nos gráficos, trazendo essa suavização nos bins dos gráficos. Outro ponto bom de se observar é a comparação com o resultado do specz e photoz

In [ ]:
plt.figure(figsize=[11,4])
plt.suptitle("redshift vs counts (no weight)", fontsize=16)

plt.hist(bpz_sem_u_band_df["zb"], np.arange(0,2,0.04), histtype="step", density=True, color="r", lw=2, label="photo-z")
plt.hist(bpz_sem_u_band_df["zspec"], np.arange(0,2,0.04), density=True, label="zspec", color="gray")

plt.xlabel("redshift", fontsize=16)
plt.ylabel("counts", fontsize=16)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend()

plt.xlim(0, 1.4)

plt.tight_layout()

## ler as pdfs e soma para depois fazer o plot em cima da pdf (numpy) ou pelo menos entender a técnica

In [ ]:
plt.figure(figsize=[10,4])
plt.suptitle("redshift vs counts (weighted)", fontsize=16)

plt.hist(bpz_df["zb"], np.arange(0,2,0.04), histtype="step", density=True, color="r", lw=2, label="photo-z", weights=df_weight)
plt.hist(bpz_df["zspec"], np.arange(0,2,0.04), density=True, label="zspec", color="gray", weights=df_weight)

plt.xlabel("redshift", fontsize=16)
plt.ylabel("counts", fontsize=16)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend()

plt.xlim(0, 2.5)
plt.xlim(0, 1.4)

plt.tight_layout()

### 2.3 - Banda i vs photoz

Para analisar os valores de redshift com valores originais da tabela principal, vamos mergear os dados em um novo df para facilitar a vida.


In [ ]:
merged_df = pd.merge(original_df, bpz_df, left_on=['number', 'field_id'], right_on=['id', 'other'], how='outer')
merged_df.head()

Pegando a magnitude usada como base, na banda i, e distribuindo com os resultados do pz, podemos ver como a maior parte se concentra entre a magnitude 18 e 24 até o photoz de valor 1. Lembrando que o valor de photoz é logaritmo.  

In [ ]:
plt.figure(figsize=[8,5])
plt.suptitle("mag i vs photoz (magdet)", fontsize=16)

plt.plot(merged_df["zb"], merged_df["i_auto"], f"k.", alpha=0.1)

plt.ylabel("mag (i)", fontsize=18)
plt.xlabel("zb", fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.xlim(0, 2)
plt.ylim(14, 25)

plt.tight_layout()

### 2.4 - Cor vs photoz

Definimos uma função para auxiliar na plotagem dos dados.

In [ ]:
def aux_color_zb(b1, b2, x_min=-1, x_max=5, y_max=5):
    plt.plot(merged_df["zb"], merged_df[f"{b1}_auto"]-merged_df[f"{b2}_auto"], f"k.", alpha=0.1)

    plt.xlabel("photo-z", fontsize=18)
    plt.ylabel(f"color ({b1}-{b2})", fontsize=18)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.ylim(x_min, x_max)
    plt.xlim(0, y_max)
    
    plt.hlines(0, 10, -1)

Plotagem inicial dos gráficos. Podemos ver que pode ser feito cortes para melhorar a visualização.

In [ ]:
plt.figure(figsize=[15,5], dpi=300)
plt.suptitle("photo-z vs color", fontsize=16)

plt.subplot(1,3,1)
aux_color_zb("g", "r")

plt.subplot(1,3,2)
aux_color_zb("r", "i")

plt.subplot(1,3,3)
aux_color_zb("i", "z")

plt.tight_layout()

In [ ]:
plt.figure(figsize=[16,5], dpi=300)
plt.suptitle("photo-z vs color", fontsize=16)

plt.subplot(1,3,1)
aux_color_zb("g", "r", -0.3, 2.5, 1)

plt.subplot(1,3,2)
aux_color_zb("r", "i", -0.3, 1.5, 1)

plt.subplot(1,3,3)
aux_color_zb("i", "z", -0.5, 1, 1)

plt.tight_layout()

## EXTRA

Se analisarmos o QA do primeiro resultado obtido rodando com a MAGAUTO foi possivel notar uma formação peculiar, a cobrinha, aqui foi feito a reprodução do mesmo comportamento para analisar esses objetos posteriormente no Target Viewer. 

In [ ]:
def plot_filtro_odds(filtro = "zb > 0", odds = "> 9"):
    plt.plot(merged_df.query(filtro)["zb"], merged_df.query(filtro)["i_auto"], f"k.", alpha=0.1)

    plt.ylabel("mag (i)", fontsize=18)
    plt.xlabel(f"zb at {odds} odds ", fontsize=18)

    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.xlim(0.25, 0.8)
    plt.ylim(18, 25)

Aqui foi feito uma evolução limitando o ODDS para cada rodada, ODDS é a porcentagem  

In [ ]:
plt.figure(figsize=[12,10])

plt.subplot(3,3,1)
plot_filtro_odds()

for i in [2, 3, 4, 5, 6, 7, 8, 9]:
    plt.subplot(3,3,i)
    plot_filtro_odds(f"odds <= 0.{11-i}", f"0.{11-i}")

plt.tight_layout()

In [ ]:
## zb = zbest
parte1 = merged_df.query("zb > 0.785 & zb < 0.8")s

plt.figure(figsize=[10,5])
plt.suptitle("mag i vs photoz (magdet)", fontsize=16)

plt.plot(parte1["zb"], parte1["i_auto"], f"k.", alpha=0.1)

plt.ylabel("mag (i)", fontsize=18)
plt.xlabel("zb", fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.xlim(0.78, 0.8)
plt.ylim(18, 25)

plt.tight_layout()

In [ ]:
parte1 = parte1.reset_index()

for i in range(10):  
    print(parte1["ra"][i], ", ", parte1["dec"][i])

# AQUI VAI FOTINHOS :D

## Métricas

In [ ]:
data_frame_bpz["erro"] = (data_frame_bpz["zbmax"] - data_frame_bpz["zbmin"])/2
data_frame_bpz.head()

In [ ]:
# photo-z
# 1-sigma photo-z error (??)
# true redshift
# field_id (from original validation set file)
# number (from original validation set file)

input_metricas = data_frame_bpz[["zb", "erro", "zspec", "id", "other"]]
input_metricas.head()

In [ ]:
file_name = "input_metricas.csv"
input_metricas.to_csv(file_name, sep=' ', header=False, index=False)